In [29]:
import pywebio
import pickle
import numpy as np
import pandas as pd
import logging
import re
from bs4 import BeautifulSoup
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

with open('variables.pkl', 'rb') as file:
      
    # create new variables from the pickle file
    nlp,stop_words,tfidf,pac = pickle.load(file)


def lemmatize(text):
    """Perform lemmatization and stopword removal in the clean text
       Returns a list of lemmas
    """
    doc = nlp(text)
    lemma_list = " ".join([str(tok.lemma_).lower() for tok in doc])
                  #if tok.is_alpha and tok.text.lower() not in stop_words])
    return lemma_list

def clean_article(x):
    x = str(x).lower().replace("\\"," ").replace("_"," ")
    x = re.sub(r"(W|w)on(\'|\’)t ", "will not ", x)
    x = re.sub(r"(C|c)an(\'|\’)t ", "can not ", x)
    x = re.sub(r"(Y|y)(\'|\’)all ", "you all ", x)
    x = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", x)
    x = re.sub(r"(I|i)(\'|\’)m ", "i am ", x)
    x = re.sub(r"(A|a)in(\'|\’)t ", "is not ", x)
    x = re.sub(r"n(\'|\’)t ", " not ", x)
    x = re.sub(r"(\'|\’)re ", " are ", x)
    x = re.sub(r"(\'|\’)s ", " is ", x)
    x = re.sub(r"(\'|\’)d ", " would ", x)
    x = re.sub(r"(\'|\’)ll ", " will ", x)
    x = re.sub(r"(\'|\’)t ", " not ", x)
    x = re.sub(r"(\'|\’)ve ", " have ", x)
    x = re.sub(r'\W+',' ',x) # Replace everything non-alpahnumeric by ' '
    x = re.sub(r'\s+',' ',x) # Replace one or more whitespaces by  ' '
    x = re.sub(r'\d+',' ',x) # Replace one or more digits by  ' '
    x = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", x) # Replace e-mails by ''
    # Replace urls by ''
    x = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , x) 
    # Replace html tags by ''
    x = BeautifulSoup(x, 'html.parser').get_text().strip()
    # Remove breaks
    x = x.replace(' br ',' ')
    # Remove standalone t and s from apostrophe removal
    #x = re.sub(r"\s*'\s*\w*",'',x)
    x = re.sub(' +', ' ',x) # Replace one or more spaces by  ' '

    return x

def demo():
    user_input = input("Give me the article you want to test:")
    clean = clean_article(user_input)
    lemma = lemmatize(clean)
    x_user = tfidf.transform([lemma])
    y = pac.predict(x_user)
    y_prop = pac._predict_proba_lr(x_user)
    if y[0] == 1:
        return "That's FAKE, I'm {:.0%} sure!".format(y_prop[0][1])
    else: 
        return "That's REAL, I'm {:.0%} sure!".format(y_prop[0][0])   

In [30]:
user_input = input("Give me the article you want to test:")
clean = clean_article(user_input)
lemma = lemmatize(clean)
x_user = tfidf.transform([lemma])
y = pac.predict(x_user)

In [31]:
print(user_input)
print(clean)
print(lemma)
print(x_user)
print(y)

This is just 1 test, to see how it works!
this is just test to see how it works
this be just test to see how it work
  (0, 29278)	0.2743486284526892
  (0, 26567)	0.7614730549318333
  (0, 26040)	0.5261968373433302
  (0, 23122)	0.2607844033848386
[1]


In [35]:
demo()

"That's REAL, I'm 55% sure!"

In [50]:
import pywebio
from pywebio.input import *
from pywebio.output import *

img = open('Presentation/FakeNewsDetector.JPG', 'rb').read()  

def demo_webio():
    while True:
             
        pywebio.output.clear()
        put_image(img, width='900px')
        user_input = pywebio.input.textarea(label="Please, copy your article here and I'll check it out for you")

        clean = clean_article(user_input)
        lemma = lemmatize(clean)
        x_user = tfidf.transform([lemma])
        y = pac.predict(x_user)
        y_prop = pac._predict_proba_lr(x_user)

        if y[0] == 1:
            pywebio.output.put_markdown("# **That's FAKE, I'm {:.0%} sure!**".format(y_prop[0][1]))
            put_image('https://media.giphy.com/media/26n6ziTEeDDbowBkQ/giphy.gif')
        else: 
            pywebio.output.put_markdown("# **That's REAL, I'm {:.0%} sure!**".format(y_prop[0][0]))
            put_image('https://media.giphy.com/media/StKjvSiOJXHOnREwnb/giphy.gif')
        if actions("Continue?",["continue", "exit"])=="exit":
            break

#pywebio.start_server(demo_webio, port=55)



In [51]:
demo_webio()
#pywebio.start_server(demo_webio, port=55)